In [143]:
using JSON
# Specify the file name and location
filename = "tiny.json"
# Read the JSON file
data = JSON.parsefile(filename)

# Extract the details from the JSON
# For example, extract all shops
linear_model = true  # Set to false if you don't want a linear model
shops = data["shops"]
println("Shops:")
println(shops)

# Extract vehicles
vehicles = data["vehicles"]
shops = data["shops"]
parameters = data["parameters"]
constraints = data["constraints"]

for vehicle in vehicles
    if linear_model
        # Set all vehicles to regular if linear_model is true
        println("Vehicle Type: regular")
    else
        # Otherwise, print the actual vehicle type
        println("Vehicle ID: ", vehicle["id"], ", Type: ", vehicle["type"])
    end
end
for constraint in constraints
    println("Constraint: ", constraint)
end
for parameter in parameters
    println("Parameter: ", parameter)
end
for shop in shops
    println("Shop: ", shop)
end



Shops:
Any[Dict{String, Any}("name" => "body", "resequencing_lag" => 1), Dict{String, Any}("name" => "paint", "resequencing_lag" => 2), Dict{String, Any}("name" => "assembly", "resequencing_lag" => 0)]
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Vehicle Type: regular
Constraint: Dict{String, Any}("max_vehicles" => 4, "cost" => 2, "min_vehicles" => 2, "id" => 1, "vehicles" => Any[1, 2, 4], "type" => "batch_size", "shop" => "body")
Constraint: Dict{String, Any}("cost" => 3, "partition" => Any[Any[1, 2], Any[3, 4, 5]], "id" => 2, "type" => "lot_change", "shop" => "paint")
Constraint: Dict{String, Any}("max_vehicles" => 2, "cost" => 2, "id" => 3, "window_size" => 3, "vehicles" => Any[1, 2, 5], "type" => "rolling_window", "shop" => "assembly")
Constraint: Dict{String, Any}("max_vehicles" => 3, "cost" => 3, "min_vehicles" => 2, "id" => 4, "vehicles" => Any[1, 2, 3], "type" => "batch_size", "shop" => "assembly")
Constraint: Dict{String, Any}("max_ve

In [144]:
cs = parameters["resequencing_cost"]
number_of_shops = length(shops)    
for shop in shops
    println("Shop: ", shop)
end
println("Number of shops: ", number_of_shops)
number_of_vehicles = length(vehicles)
println("number_of_vehicles, ",number_of_vehicles)

Shop: Dict{String, Any}("name" => "body", "resequencing_lag" => 1)
Shop: Dict{String, Any}("name" => "paint", "resequencing_lag" => 2)
Shop: Dict{String, Any}("name" => "assembly", "resequencing_lag" => 0)
Number of shops: 3
number_of_vehicles, 5


In [145]:

# Initialize an empty dictionary to store the mapping
shop_map = Dict{String, Vector{Any}}()
""" In shop_map, every shop name is a key, and the value is a vector of constraints associated with that shop. """
lag_map = Dict{String, Integer}()
""" In shop_map, every shop name is a key, and the value is a vector of constraints associated with that shop. """
# Populate the shop_map with shop names as keys and empty arrays as values
for shop in shops
    shop_map[shop["name"]] = []
    lag_map[shop["name"]] = shop["resequencing_lag"]
    print(lag_map[shop["name"]])
end
n = number_of_vehicles
println("n: ", n)
# Initialize counters or variables if needed

# Assume `model` is defined somewhere before this loop (like in JuMP)
# Example: model = Model(Ipopt.Optimizer)

# Iterate through constraints and associate them with the corresponding shop in shop_map

for constraint in constraints
    name = constraint["shop"]
    if haskey(shop_map, name)
        push!(shop_map[name], constraint)
    else
        println("Warning: Shop name $name in constraints does not exist in shops.")
    end
end

(lag_map)


120n: 5


Dict{String, Integer} with 3 entries:
  "body"     => 1
  "assembly" => 0
  "paint"    => 2

In [146]:
# Function to perturb a solution (for example, swapping two elements)
function perturb_solution_big(perm)
    new_perm = copy(perm)
    idx1, idx2 = rand(1:length(perm), 2)
    # Swap two random elements
    new_perm[idx1], new_perm[idx2] = new_perm[idx2], new_perm[idx1]
    return new_perm
end

function perturb_solution(perm)
    new_perm = copy(perm)
    # Randomly select an index for the swap, ensuring it's not the last index
    idx = rand(1:length(perm) - 1)
    # Swap the element at idx with the element at idx + 1
    new_perm[idx], new_perm[idx + 1] = new_perm[idx + 1], new_perm[idx]
    return new_perm
end



perturb_solution (generic function with 1 method)

In [147]:
function calculate_resequencing_cost(permutations, Δs, cs)
    # Initialize the resequencing cost
    total_cost = 0

    # Number of shops (S)
    S = length(permutations)
    
    # Iterate over each consecutive pair of shops
    for s in 1:(S-1)
        # Get the current shop and the next shop
        perm_s = permutations[s]
        perm_s_next = permutations[s+1]
        
        # Iterate through each vehicle
        for v in 1:length(perm_s)
            # Get the position of vehicle v in the current shop (s) and the next shop (s+1)
            t_s = findfirst(isequal(v), perm_s)  # Position of v in shop s
            t_s_next = findfirst(isequal(v), perm_s_next)  # Position of v in shop s+1
            
            # Handle cases where the position is not found (i.e., t_s or t_s_next is nothing)
            if isnothing(t_s) || isnothing(t_s_next)
                continue  # Skip this vehicle if the position is not found
            end
            
            # Access Δs based on the shop index `s`
            Δs_value = Δs[s]  # Δs indexed by s
            
            # Ensure Δs_value is not Nothing or handle the case if it's out of expected bounds
            if isnothing(Δs_value)
                continue  # Skip if the Δs_value is not available
            end

            # Check if the vehicle satisfies the constraint
            if t_s_next >= t_s - Δs_value  # Use the Δs value safely
                # No penalty if the constraint is satisfied
                continue
            else
                # Calculate penalty if the constraint is violated
                penalty = cs * max(0, t_s - Δs_value - t_s_next)
                total_cost += penalty
            end
        end
    end
    
    return total_cost
end


calculate_resequencing_cost (generic function with 1 method)

In [148]:
function calculate_cost_origin(perm, Cl, Ul, Vr, Mr, wr, cr, mb, Mb, cb, Vb)
    total_cost = 0

    # Lot change constraints (Cl and Ul)
    if !isempty(Cl) && !isempty(Ul)
        for (i, l) in enumerate(Cl)
            for s in l
                for t in 1:(length(perm)-1)
                    # Find the group for perm[t] and perm[t+1]
                    group_t = findfirst(g -> perm[t] in g, Ul[i])
                    group_t1 = findfirst(g -> perm[t+1] in g, Ul[i])
    
    
                    # Check if consecutive vehicles belong to different groups
                    if group_t != group_t1
                        total_cost += s  # Add penalty associated with the lot change
                    end
                end
            end
        end
    end
    

    # Rolling window constraints (Vr, Mr, wr, cr)
    if !isempty(Vr) && !isempty(Mr) && !isempty(wr) && !isempty(cr)
        for (r_idx, r) in enumerate(Vr)
            max_violations = 0
            for t in 1:(length(perm) - wr[r_idx] + 1)
                window = perm[t:(t+wr[r_idx]-1)]
                violation_count = sum(v -> v in r, window)
                if violation_count > Mr[r_idx]
                    max_violations += (violation_count - Mr[r_idx])^2 * cr[r_idx]
                end
            end
            total_cost += max_violations
        end
    end

    # Batch size constraints (Vb, mb, Mb, cb)
    if !isempty(Vb) && !isempty(mb) && !isempty(Mb) && !isempty(cb)
        for (b_idx, b) in enumerate(Vb)
            max_violations = 0
            batch_start = 1
            while batch_start <= length(perm)
                batch_end = batch_start
                while batch_end < length(perm) && perm[batch_end+1] in b
                    batch_end += 1
                end
                batch_size = batch_end - batch_start + 1
                if batch_size < mb[b_idx] || batch_size > Mb[b_idx]
                    max_violations += (max(0, mb[b_idx] - batch_size, batch_size - Mb[b_idx]))^2 * cb[b_idx]
                end
                batch_start = batch_end + 1
            end
            total_cost += max_violations
        end
    end

    return total_cost
end

calculate_cost_origin (generic function with 1 method)

In [149]:
function calculate_cost(perms, Cl, Ul, Vr, Mr, wr, cr, mb, Mb, cb, Vb,Δs,cs)
    cost = calculate_resequencing_cost(perms, Δs, cs)
    for i  in length(perms) 
        cost+= calculate_cost_origin(perms[i], Cl[i], Ul[i], Vr[i], Mr[i], wr[i], cr[i], mb[i], Mb[i], cb[i], Vb[i])
    end
    return cost
end


calculate_cost (generic function with 1 method)

In [150]:
function simulated_annealing(perms, Cl, Ul, Vr, Mr, wr, cr, mb, Mb, cb, Vb,Δs,cs; T0=1000, alpha=0.7, max_iter=10000000)
    # Initialize variables
    current_cost = calculate_cost(perms, Cl, Ul, Vr, Mr, wr, cr, mb, Mb, cb, Vb, Δs, cs)  # Calculate cost of the initial solution
    best_perms = perms  # Best solution found
    best_cost = current_cost  # Best cost found
    T = T0  # Initial temperature
    counter = 0
    
    for iter in 1:max_iter
        if (T < 1e-50)
            break
        end
        neighbor_costs = []
        neighbor_perms = []
        # Generate a set of neighboring solutions (perturbations)
        for i in 1:length(perms)
            perm_copy = perms
            perm = perms[i]
            perturbed_perm = perturb_solution(perm)
            perm_copy[i] = perturbed_perm
            perturbed_cost = calculate_cost(perm_copy, Cl, Ul, Vr, Mr, wr, cr, mb, Mb, cb, Vb, Δs, cs)
            push!(neighbor_perms,perm_copy)
            push!(neighbor_costs,perturbed_cost)
        end
        # Find the best neighboring solution
        min_cost_idx = argmin(neighbor_costs)
        best_neighbor = neighbor_perms[min_cost_idx]
        best_neighbor_cost = neighbor_costs[min_cost_idx]

        # If the best neighboring solution is better, update the current solution
        if best_neighbor_cost < current_cost
            counter = 0
            perms = best_neighbor
            current_cost = best_neighbor_cost

            # If the new solution is the best, update the best solution
            if best_neighbor_cost < best_cost
                best_perms = best_neighbor
                best_cost = best_neighbor_cost
            end
        else
            # If the new solution is worse, accept it with a certain probability based on temperature
            counter += 1
            if counter < 10
                acceptance_probability = exp(-(best_neighbor_cost - current_cost) / T)
                if rand() < acceptance_probability
                    perms = best_neighbor
                    current_cost = best_neighbor_cost
                end
            else
                # Larger perturbation for worse solutions
                acceptance_probability = exp(-(best_neighbor_cost - current_cost) / T)
                if rand() < acceptance_probability
                    perms = perturb_solution_big(perms)
                    current_cost = calculate_cost(perms, Cl, Ul, Vr, Mr, wr, cr, mb, Mb, cb, Vb, Δs, cs)
                end
            end
        end

        # Reduce the temperature (cooling schedule)
        T *= alpha

        # Optionally print progress
        if iter % 1000 == 0
            println("Iteration: $iter, Best Cost: $best_cost, Temperature: $T")
        end
    end

    return best_perms, best_cost
end


simulated_annealing (generic function with 2 methods)

In [151]:
# Initialize result and Δs lists
result = []
Δs = []

# Initialize global constraint lists for all shops
Cl = []          # Cost list (Ints)
Ul = []  # Partitions (Array of arrays of Ints)
Vr = []  # Vehicles (Array of arrays of Ints)
Mr = []          # Max vehicles (Ints)
wr = []          # Window size (Ints)
cr = []          # Cost (Ints)
mb = []          # Min vehicles (Ints)
Mb = []          # Max vehicles (Ints)
cb = []          # Cost (Ints)
Vb = []  # Vehicles (Array of Ints)

# Loop through each shop and its constraints
for (shop, constraints) in shop_map
    println("shop: ", shop)
    
    # Initialize shop-specific constraint lists
    Cl0 = Int[]          # Cost list (Ints)
    Ul0 = Vector{Vector{Int}}[]  # Partitions (Array of arrays of Ints)
    Vr0 = Vector{Int}[]  # Vehicles (Array of arrays of Ints)
    Mr0 = Int[]          # Max vehicles (Ints)
    wr0 = Int[]          # Window size (Ints)
    cr0 = Int[]          # Cost (Ints)
    mb0 = Int[]          # Min vehicles (Ints)
    Mb0 = Int[]          # Max vehicles (Ints)
    cb0 = Int[]          # Cost (Ints)
    Vb0 = Vector{Int}[]  # Vehicles (Array of Ints)
    
    # Add the lag for the current shop
    push!(Δs, lag_map[shop])
    
    # Loop through constraints and separate them by type
    for constraint in constraints
        if constraint["type"] == "lot_change"
            cl = constraint["cost"]
            Ul_0 = constraint["partition"]
            push!(Ul0, Ul_0)  # Ensure Ul_0 is an array of Int
            push!(Cl0, cl)
        end

        if constraint["type"] == "rolling_window"
            Vr_0 = constraint["vehicles"]
            Mr_0 = constraint["max_vehicles"]
            wr_0 = constraint["window_size"]
            cr_0 = constraint["cost"]
            push!(Vr0, Vr_0)  # Ensure Vr_0 is an array of Int
            push!(Mr0, Mr_0)
            push!(wr0, wr_0)
            push!(cr0, cr_0)
        end

        if constraint["type"] == "batch_size"
            mb_0 = constraint["min_vehicles"]
            Mb_0 = constraint["max_vehicles"]
            Vb_0 = constraint["vehicles"]
            cb_0 = constraint["cost"]
            push!(mb0, mb_0)
            push!(Mb0, Mb_0)
            push!(Vb0, Vb_0)  # Ensure Vb_0 is an array of Int
            push!(cb0, cb_0)
        end
    end

    # Push all constraint lists (even if they are empty) to the global lists
    push!(Cl, Cl0)
    push!(Ul, Ul0)
    push!(Vr, Vr0)
    push!(Mr, Mr0)
    push!(wr, wr0)
    push!(cr, cr0)
    push!(mb, mb0)
    push!(Mb, Mb0)
    push!(cb, cb0)
    push!(Vb, Vb0)
end
perms = [[i for i in 1:n] for k in 1:3]
println(perms)
println(length(perms))
simulated_annealing(perms, Cl, Ul, Vr, Mr, wr, cr, mb, Mb, cb, Vb,Δs,cs; T0=1000, alpha=0.7, max_iter=10000000)

shop: body
shop: assembly
shop: paint
[[1, 2, 3, 4, 5], [1, 2, 3, 4, 5], [1, 2, 3, 4, 5]]
3


([[2, 1, 5, 3, 4], [5, 1, 3, 2, 4], [5, 3, 2, 1, 4]], 3)

3-element Vector{Any}:
 1
 0
 2

In [153]:
Δs

3-element Vector{Any}:
 1
 0
 2